In [3]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [6]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH,Alagoas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagôas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Educacao,Alagoas - Consumo de Cimento (t)
0,2014,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.664799,14.169213,5.676413e+08,2.779340e+09,1.954361e+10,...,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262,650.702685,0.754811,0.623692,818.645143
1,2015,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.677623,14.659464,5.179392e+08,2.561731e+09,1.327596e+10,...,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007,525.071908,0.768560,0.640614,676.300114
2,2016,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.678679,14.776623,4.524144e+08,2.516859e+09,1.011370e+10,...,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435,350.210234,0.768520,0.643852,538.586743
3,2017,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.678559,14.347557,4.213689e+08,2.497710e+09,8.456309e+09,...,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980,239.571638,0.767840,0.644657,362.791000
4,2018,3.129885e+07,1.302198e+06,7.848307,2.472535e+07,0.678519,14.104975,4.166580e+08,2.797145e+09,7.937760e+09,...,1005.194377,1515.339234,25.890613,21.302886,77.069840,2.720044,257.108625,0.766553,0.645625,340.278000
5,2019,3.114242e+07,1.306516e+06,7.842878,2.470072e+07,0.677567,13.904156,4.462625e+08,3.484252e+09,1.091282e+10,...,1004.925660,1512.865070,26.413908,21.285962,74.182188,2.888728,298.396670,0.764603,0.645137,395.359000
6,2020,3.076167e+07,1.314933e+06,7.822101,2.461648e+07,0.676709,13.726047,5.489412e+08,4.114911e+09,1.625333e+10,...,1004.784156,1488.758698,26.343333,21.178201,74.384289,2.840934,309.942452,0.762022,0.645331,581.479000
7,2021,3.012288e+07,1.317971e+06,7.792038,2.449424e+07,0.675137,13.450561,6.638811e+08,4.421848e+09,1.933031e+10,...,1004.790286,1463.198430,25.912167,21.228201,78.250990,2.687209,302.221566,0.759842,0.643997,576.578000
8,2022,2.935146e+07,1.321357e+06,7.755110,2.434266e+07,0.674331,13.470373,7.010332e+08,4.533216e+09,1.997888e+10,...,1004.840962,1460.802556,25.797702,21.256957,78.638550,2.677640,294.171874,0.760918,0.640815,547.622000


In [7]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH,Alagoas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagôas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Educacao
0,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.664799,14.169213,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262,650.702685,0.754811,0.623692
1,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.677623,14.659464,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007,525.071908,0.768560,0.640614
2,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.678679,14.776623,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435,350.210234,0.768520,0.643852
3,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.678559,14.347557,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980,239.571638,0.767840,0.644657
4,3.129885e+07,1.302198e+06,7.848307,2.472535e+07,0.678519,14.104975,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,16.279153,1005.194377,1515.339234,25.890613,21.302886,77.069840,2.720044,257.108625,0.766553,0.645625
5,3.114242e+07,1.306516e+06,7.842878,2.470072e+07,0.677567,13.904156,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,14.391737,1004.925660,1512.865070,26.413908,21.285962,74.182188,2.888728,298.396670,0.764603,0.645137
6,3.076167e+07,1.314933e+06,7.822101,2.461648e+07,0.676709,13.726047,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,14.238466,1004.784156,1488.758698,26.343333,21.178201,74.384289,2.840934,309.942452,0.762022,0.645331


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    676.300114
1    538.586743
2    362.791000
3    340.278000
4    395.359000
5    581.479000
6    576.578000
7    547.622000
8           NaN
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH,Alagoas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagôas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Educacao
0,-2.006396,2.061261,-2.024857,-1.720367,-2.227181,-0.513682,1.779974,0.019134,2.003355,-1.880579,...,0.553736,0.702249,-0.721361,-0.770299,-1.135674,0.566246,-0.748080,1.749922,-2.144656,-2.186118
1,-0.310376,0.247672,-0.239174,0.567855,0.332331,1.082366,0.870373,-0.621387,0.401153,0.111653,...,0.092476,0.915397,-0.268376,-0.515587,-0.969910,0.317382,-0.379154,0.916734,0.707948,0.002331
2,0.599416,-0.247427,0.081150,0.747102,0.543224,1.463787,-0.328802,-0.753466,-0.407217,1.154682,...,0.609613,0.583195,-0.905278,-0.624446,-0.739947,0.501400,-0.150580,-0.242954,0.699651,0.421047
3,0.999124,-0.876203,0.964268,1.193178,0.519262,0.066932,-0.896969,-0.809829,-0.830898,0.955301,...,0.933697,0.388999,-0.866719,-0.659236,0.366745,0.882859,-0.875603,-0.976712,0.558572,0.525118
4,0.721446,-0.874111,0.895528,0.067501,0.511267,-0.722811,-0.983185,0.071542,-0.963455,0.100493,...,-0.083722,-0.653116,1.488793,0.572415,1.326877,-0.139928,0.030607,-0.860407,0.291501,0.650407
5,-0.003214,-0.311192,0.323085,-0.855270,0.321097,-1.376592,-0.441390,2.094006,-0.202939,-0.441549,...,-2.105801,-1.936723,1.272941,1.997154,1.151909,-2.127960,2.122811,-0.586583,-0.113017,0.587216


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    676.300114
1    538.586743
2    362.791000
3    340.278000
4    395.359000
5    581.479000
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH,Alagoas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagôas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Educacao
6,-1.360399,0.697668,-1.412698,-2.044801,0.138585,-1.456381,1.16956,2.035192,0.985175,-0.705937,...,-1.595006,-1.721117,-0.733324,1.380453,0.035006,-1.471828,1.226248,-0.463643,-0.58323,0.552315


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    576.578
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [23]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [24]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [25]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3954073949, 2232992384, 2042969154, 1172275919, 41463769, 2544414324, 1211547965, 3352882038, 1048290899, 1211713496]


Step: 0 ___________________________________________
Epoch 816: early stopping
val_loss: 18736.98046875
winner_seed: 3954073949


Step: 1 ___________________________________________
Epoch 1353: early stopping
val_loss: 8660.3671875
winner_seed: 2232992384


Step: 2 ___________________________________________
Epoch 848: early stopping
val_loss: 8481.84375
winner_seed: 2042969154


Step: 3 ___________________________________________
Epoch 575: early stopping
val_loss: 0.022342804819345474
winner_seed: 1172275919


Step: 4 ___________________________________________
Epoch 1352: early stopping
val_loss: 8660.3623046875


Step: 5 ___________________________________________
Epoch 1873: early stopping
val_loss: 8660.56640625


Step: 6 ___________________________________________
Epoch 905: early stopping
val_loss: 5995.9755859375


Step: 7 ____________________________________

In [26]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 301ms/step - loss: 248232.6719 - val_loss: 330939.2500
Epoch 2/5000
1/1 [==============================] - 0s 25ms/step - loss: 236299.7500 - val_loss: 395548.6250
Epoch 3/5000
1/1 [==============================] - 0s 25ms/step - loss: 217521.1406 - val_loss: 183585.5156
Epoch 4/5000
1/1 [==============================] - 0s 26ms/step - loss: 210114.8125 - val_loss: 424527.0938
Epoch 5/5000
1/1 [==============================] - 0s 33ms/step - loss: 199164.8125 - val_loss: 436011.8750
Epoch 6/5000
1/1 [==============================] - 0s 34ms/step - loss: 216089.2031 - val_loss: 164511.7344
Epoch 7/5000
1/1 [==============================] - 0s 27ms/step - loss: 200225.5156 - val_loss: 151713.2031
Epoch 8/5000
1/1 [==============================] - 0s 26ms/step - loss: 176935.2500 - val_loss: 164883.0156
Epoch 9/5000
1/1 [==============================] - 0s 26ms/step - loss: 172351.1719 - val_loss: 146110.3906
Epoch 10/5000
1/1 

1/1 [==============================] - 0s 25ms/step - loss: 13483.8076 - val_loss: 13.9232
Epoch 78/5000
1/1 [==============================] - 0s 26ms/step - loss: 13117.5752 - val_loss: 31.1557
Epoch 79/5000
1/1 [==============================] - 0s 27ms/step - loss: 12765.7549 - val_loss: 54.3753
Epoch 80/5000
1/1 [==============================] - 0s 27ms/step - loss: 12427.7139 - val_loss: 82.9852
Epoch 81/5000
1/1 [==============================] - 0s 27ms/step - loss: 12102.8447 - val_loss: 116.4249
Epoch 82/5000
1/1 [==============================] - 0s 25ms/step - loss: 11790.5791 - val_loss: 154.1787
Epoch 83/5000
1/1 [==============================] - 0s 28ms/step - loss: 11490.3662 - val_loss: 195.7506
Epoch 84/5000
1/1 [==============================] - 0s 29ms/step - loss: 11201.6836 - val_loss: 240.6967
Epoch 85/5000
1/1 [==============================] - 0s 25ms/step - loss: 10924.0283 - val_loss: 288.5934
Epoch 86/5000
1/1 [==============================] - 0s 25ms/ste

1/1 [==============================] - 0s 27ms/step - loss: 3735.2385 - val_loss: 2399.0984
Epoch 155/5000
1/1 [==============================] - 0s 27ms/step - loss: 3701.9412 - val_loss: 2392.5259
Epoch 156/5000
1/1 [==============================] - 0s 26ms/step - loss: 3669.3679 - val_loss: 2385.3184
Epoch 157/5000
1/1 [==============================] - 0s 26ms/step - loss: 3637.5039 - val_loss: 2377.4968
Epoch 158/5000
1/1 [==============================] - 0s 26ms/step - loss: 3606.3196 - val_loss: 2369.0940
Epoch 159/5000
1/1 [==============================] - 0s 26ms/step - loss: 3575.8020 - val_loss: 2360.1367
Epoch 160/5000
1/1 [==============================] - 0s 27ms/step - loss: 3545.9280 - val_loss: 2350.6282
Epoch 161/5000
1/1 [==============================] - 0s 27ms/step - loss: 3516.6807 - val_loss: 2340.6130
Epoch 162/5000
1/1 [==============================] - 0s 25ms/step - loss: 3488.0422 - val_loss: 2330.1008
Epoch 163/5000
1/1 [==============================] 

1/1 [==============================] - 0s 27ms/step - loss: 2395.3206 - val_loss: 1234.7692
Epoch 231/5000
1/1 [==============================] - 0s 27ms/step - loss: 2386.8284 - val_loss: 1219.7419
Epoch 232/5000
1/1 [==============================] - 0s 26ms/step - loss: 2378.4712 - val_loss: 1204.8450
Epoch 233/5000
1/1 [==============================] - 0s 26ms/step - loss: 2370.2429 - val_loss: 1190.0647
Epoch 234/5000
1/1 [==============================] - 0s 28ms/step - loss: 2362.1460 - val_loss: 1175.4133
Epoch 235/5000
1/1 [==============================] - 0s 31ms/step - loss: 2354.1731 - val_loss: 1160.8860
Epoch 236/5000
1/1 [==============================] - 0s 28ms/step - loss: 2346.3250 - val_loss: 1146.4862
Epoch 237/5000
1/1 [==============================] - 0s 27ms/step - loss: 2338.5986 - val_loss: 1132.2133
Epoch 238/5000
1/1 [==============================] - 0s 29ms/step - loss: 2330.9927 - val_loss: 1118.0704
Epoch 239/5000
1/1 [==============================] 

1/1 [==============================] - 0s 26ms/step - loss: 2010.2369 - val_loss: 437.0254
Epoch 308/5000
1/1 [==============================] - 0s 29ms/step - loss: 2007.6299 - val_loss: 430.8209
Epoch 309/5000
1/1 [==============================] - 0s 50ms/step - loss: 2005.0620 - val_loss: 424.6960
Epoch 310/5000
1/1 [==============================] - 0s 30ms/step - loss: 2002.5321 - val_loss: 418.6550
Epoch 311/5000
1/1 [==============================] - 0s 25ms/step - loss: 2000.0406 - val_loss: 412.6944
Epoch 312/5000
1/1 [==============================] - 0s 25ms/step - loss: 1997.5868 - val_loss: 406.8134
Epoch 313/5000
1/1 [==============================] - 0s 27ms/step - loss: 1995.1698 - val_loss: 401.0089
Epoch 314/5000
1/1 [==============================] - 0s 27ms/step - loss: 1992.7894 - val_loss: 395.2874
Epoch 315/5000
1/1 [==============================] - 0s 33ms/step - loss: 1990.4438 - val_loss: 389.6407
Epoch 316/5000
1/1 [==============================] - 0s 31ms

1/1 [==============================] - 0s 27ms/step - loss: 1890.6442 - val_loss: 141.6818
Epoch 385/5000
1/1 [==============================] - 0s 32ms/step - loss: 1889.8278 - val_loss: 139.5928
Epoch 386/5000
1/1 [==============================] - 0s 33ms/step - loss: 1889.0243 - val_loss: 137.5366
Epoch 387/5000
1/1 [==============================] - 0s 33ms/step - loss: 1888.2323 - val_loss: 135.5070
Epoch 388/5000
1/1 [==============================] - 0s 31ms/step - loss: 1887.4521 - val_loss: 133.5080
Epoch 389/5000
1/1 [==============================] - 0s 27ms/step - loss: 1886.6841 - val_loss: 131.5406
Epoch 390/5000
1/1 [==============================] - 0s 26ms/step - loss: 1885.9268 - val_loss: 129.5990
Epoch 391/5000
1/1 [==============================] - 0s 28ms/step - loss: 1885.1815 - val_loss: 127.6870
Epoch 392/5000
1/1 [==============================] - 0s 32ms/step - loss: 1884.4469 - val_loss: 125.8028
Epoch 393/5000
1/1 [==============================] - 0s 32ms

Epoch 462/5000
1/1 [==============================] - 0s 28ms/step - loss: 1852.9136 - val_loss: 44.1839
Epoch 463/5000
1/1 [==============================] - 0s 26ms/step - loss: 1852.6615 - val_loss: 43.5258
Epoch 464/5000
1/1 [==============================] - 0s 33ms/step - loss: 1852.4130 - val_loss: 42.8784
Epoch 465/5000
1/1 [==============================] - 0s 35ms/step - loss: 1852.1685 - val_loss: 42.2397
Epoch 466/5000
1/1 [==============================] - 0s 30ms/step - loss: 1851.9272 - val_loss: 41.6105
Epoch 467/5000
1/1 [==============================] - 0s 24ms/step - loss: 1851.6899 - val_loss: 40.9908
Epoch 468/5000
1/1 [==============================] - 0s 26ms/step - loss: 1851.4564 - val_loss: 40.3803
Epoch 469/5000
1/1 [==============================] - 0s 39ms/step - loss: 1851.2261 - val_loss: 39.7791
Epoch 470/5000
1/1 [==============================] - 0s 38ms/step - loss: 1850.9994 - val_loss: 39.1870
Epoch 471/5000
1/1 [==============================] - 0

1/1 [==============================] - 0s 29ms/step - loss: 1841.2588 - val_loss: 13.6851
Epoch 541/5000
1/1 [==============================] - 0s 37ms/step - loss: 1841.1807 - val_loss: 13.4809
Epoch 542/5000
1/1 [==============================] - 0s 33ms/step - loss: 1841.1041 - val_loss: 13.2795
Epoch 543/5000
1/1 [==============================] - 0s 31ms/step - loss: 1841.0287 - val_loss: 13.0814
Epoch 544/5000
1/1 [==============================] - 0s 36ms/step - loss: 1840.9542 - val_loss: 12.8857
Epoch 545/5000
1/1 [==============================] - 0s 38ms/step - loss: 1840.8810 - val_loss: 12.6936
Epoch 546/5000
1/1 [==============================] - 0s 32ms/step - loss: 1840.8086 - val_loss: 12.5042
Epoch 547/5000
1/1 [==============================] - 0s 36ms/step - loss: 1840.7374 - val_loss: 12.3172
Epoch 548/5000
1/1 [==============================] - 0s 35ms/step - loss: 1840.6674 - val_loss: 12.1332
Epoch 549/5000
1/1 [==============================] - 0s 27ms/step - l

In [27]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 32ms/step
[576.578](test_target) - [[605.28577]](prediction) = 28.707766601562525


In [28]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [29]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH,Alagoas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagôas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Educacao
0,-1.327924,1.384573,-1.400057,-1.41121,-1.410904,-1.390649,1.165029,1.396038,1.335266,-1.330895,...,0.583110,-0.228234,-0.335115,-1.279548,-1.153465,0.992032,-1.308662,1.152715,-1.414209,-1.398314
1,0.242699,-0.442849,0.527172,0.62582,0.621712,0.472677,0.111769,-0.502296,-0.264151,0.251276,...,-1.407345,1.322807,1.357420,1.161375,-0.131885,-1.368887,0.190055,0.133175,0.710194,0.516022
2,1.085225,-0.941724,0.872886,0.78539,0.789192,0.917972,-1.276798,-0.893743,-1.071116,1.079619,...,0.824235,-1.094573,-1.022305,0.118172,1.285350,0.376855,1.118607,-1.285890,0.704015,0.882292


train_target:


,Alagoas - Consumo de Cimento (t)
0,676.300114
1,538.586743
2,362.791000


test_input:


...

,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH,Alagoas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagôas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Educacao
6,-1.360399,0.697668,-1.412698,-2.044801,0.138585,-1.456381,1.16956,2.035192,0.985175,-0.705937,...,-1.595006,-1.721117,-0.733324,1.380453,0.035006,-1.471828,1.226248,-0.463643,-0.58323,0.552315


test_target:


,Alagoas - Consumo de Cimento (t)
6,576.578


Epoch 575: early stopping
1/1 [==============================] - 0s 32ms/step
Error: 28.707766601562525




[362.78875732421875, 351.53375244140625, 367.8175964355469, 605.2857666015625]

In [33]:
display(targets)
display(predictions)

[340.278, 395.359, 581.479, 576.578]

[362.78875732421875, 351.53375244140625, 367.8175964355469, 605.2857666015625]

In [34]:
mae = mean_absolute_error(predictions, targets)
mae

77.17629376220704

In [35]:
porcentage = mae/np.mean(targets)
porcentage

0.1630174542712963